# Hacemos las importaciones oportunas

In [1]:
import asyncio

from joblib import Parallel, delayed

import requests as req
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import Select

import time
import multiprocessing as mp

import pandas as pd
pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

from tqdm.notebook import tqdm

#from webdriver_manager.chrome import ChromeDriverManager
#PATH=ChromeDriverManager().install()

PATH=r'Chromedriver\chromedriver'

import warnings
warnings.filterwarnings('ignore')

# Abrir Archivo csv de los stats de los GM's (fuente: Kaggle)

In [2]:
stats_original = pd.read_csv(r'GM_players_statistics.csv')
stats = stats_original.copy()
stats.head()

,Unnamed: 0,name,username,profile,user_id,title,fide,country,followers,joined,last_online,current_rapid_rating,highest_rapid_rating,highest_rapid_date,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,highest_blitz_date,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,highest_bullet_date,bullet_win,bullet_draw,bullet_loss,highest_tactics_rating,highest_tactics_date,highest_puzzle_rush_score,is_streamer,status,league
0,0,Komodo Engine,komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,494,07/11/2015,17/09/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400,07/11/2015,NaN,False,premium,NaN
1,1,Vojtěch Plát,vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,360,11/08/2017,13/07/2023,2383.0,2672.0,03/05/2020,12.0,5.0,6.0,2823.0,2914.0,14/07/2020,976.0,280.0,847.0,2661.0,2750.0,27/09/2021,32.0,8.0,47.0,3251,15/09/2021,49.0,False,premium,Bronze
2,2,PlayMagnus Carlsen,playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,199,05/11/2014,08/12/2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400,05/11/2014,NaN,False,premium,NaN
3,3,Magnus Carlsen,magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,142482,27/08/2010,15/07/2023,2925.0,2977.0,07/02/2023,43.0,23.0,8.0,3287.0,3322.0,21/06/2023,1402.0,292.0,334.0,3249.0,3390.0,28/03/2023,662.0,98.0,246.0,400,27/08/2010,NaN,False,premium,Elite
4,4,Fabiano Caruana,fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,14445,17/03/2013,16/07/2023,2794.0,3045.0,01/09/2019,122.0,103.0,46.0,3125.0,3136.0,28/05/2023,1012.0,194.0,322.0,3030.0,3113.0,30/11/2021,713.0,65.0,203.0,3519,14/06/2023,64.0,False,premium,Crystal


# Limpieza csv

In [3]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 1508 non-null   int64  
 1   name                       1241 non-null   object 
 2   username                   1508 non-null   object 
 3   profile                    1508 non-null   object 
 4   user_id                    1508 non-null   int64  
 5   title                      1508 non-null   object 
 6   fide                       913 non-null    float64
 7   country                    1508 non-null   object 
 8   followers                  1508 non-null   int64  
 9   joined                     1508 non-null   object 
 10  last_online                1508 non-null   object 
 11  current_rapid_rating       1055 non-null   float64
 12  highest_rapid_rating       997 non-null    float64
 13  highest_rapid_date         997 non-null    objec

In [4]:
stats = stats.drop(['is_streamer', 'status', 'league', 'highest_puzzle_rush_score', 'highest_tactics_date', 'highest_tactics_rating', 'joined', 'followers', 'last_online'], axis=1)
stats

,Unnamed: 0,name,username,profile,user_id,title,fide,country,current_rapid_rating,highest_rapid_rating,highest_rapid_date,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,highest_blitz_date,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,highest_bullet_date,bullet_win,bullet_draw,bullet_loss
0,0,Komodo Engine,komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Vojtěch Plát,vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,2383.0,2672.0,03/05/2020,12.0,5.0,6.0,2823.0,2914.0,14/07/2020,976.0,280.0,847.0,2661.0,2750.0,27/09/2021,32.0,8.0,47.0
2,2,PlayMagnus Carlsen,playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Magnus Carlsen,magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,2925.0,2977.0,07/02/2023,43.0,23.0,8.0,3287.0,3322.0,21/06/2023,1402.0,292.0,334.0,3249.0,3390.0,28/03/2023,662.0,98.0,246.0
4,4,Fabiano Caruana,fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,2794.0,3045.0,01/09/2019,122.0,103.0,46.0,3125.0,3136.0,28/05/2023,1012.0,194.0,322.0,3030.0,3113.0,30/11/2021,713.0,65.0,203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,1503,Stanojoski Zvonko,zona1,https://www.chess.com/member/Zona1,13526852,GM,NaN,North Macedonia,NaN,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,02/05/2016,50.0,9.0,27.0,2102.0,2083.0,27/09/2014,13.0,1.0,7.0
1504,1504,Dmitry Gurevich,zq1,https://www.chess.com/member/Zq1,4042329,GM,NaN,United States,2375.0,2555.0,30/11/2020,7.0,2.0,6.0,2415.0,2675.0,13/01/2022,3293.0,538.0,2521.0,2400.0,2594.0,04/04/2019,496.0,47.0,402.0
1505,1505,Alexander Zubov,zubov_alexander,https://www.chess.com/member/Zubov_Alexander,83663714,GM,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,30/08/2021,338.0,22.0,96.0,NaN,NaN,NaN,NaN,NaN,NaN
1506,1506,Alexander Zubov,zubov_on_youtube,https://www.chess.com/member/Zubov_On_YouTube,148025331,GM,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,25/07/2021,138.0,3.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1508 non-null   int64  
 1   name                   1241 non-null   object 
 2   username               1508 non-null   object 
 3   profile                1508 non-null   object 
 4   user_id                1508 non-null   int64  
 5   title                  1508 non-null   object 
 6   fide                   913 non-null    float64
 7   country                1508 non-null   object 
 8   current_rapid_rating   1055 non-null   float64
 9   highest_rapid_rating   997 non-null    float64
 10  highest_rapid_date     997 non-null    object 
 11  rapid_win              1055 non-null   float64
 12  rapid_draw             1055 non-null   float64
 13  rapid_loss             1055 non-null   float64
 14  current_blitz_rating   1388 non-null   float64
 15  high

In [6]:
mp.cpu_count() # nº de núcleos

4

In [7]:
stats['fide'].value_counts()

0.0       199
2500.0     22
2600.0     19
2550.0     14
2650.0     11
         ... 
2697.0      1
2698.0      1
2705.0      1
2706.0      1
3411.0      1
Name: fide, Length: 310, dtype: int64

# 1. Scrapeo de datos (rating)

## 1.1 Usando beautiful soup

In [8]:
url = 'https://2700chess.com/?per-page=100'

html = req.get(url).text

sopa = bs(html, 'html.parser')

In [9]:
#divs = for e in sopa.find_all('table')
#divs

In [10]:
#table = e.text for e in divs.find_all('table')

## 1.2 Usando Selenium

In [11]:
driver = webdriver.Chrome(PATH)

driver.get(url)

In [12]:
#cookies
aceptar = driver.find_element(By.XPATH, '//*[@id="accept-choices"]')

aceptar.click()

In [13]:
table = driver.find_element(By.XPATH, '//*[@id="live-ratings-table"]')
table

<selenium.webdriver.remote.webelement.WebElement (session="c9b255e0e983dc21e0c31091fd0a5f60", element="28C66D9D53EBD75497676C7D6DADCD7C_element_103")>

In [14]:
rows = [[j.text for j in i.find_elements(By.TAG_NAME,'td')]
for i in driver.find_element(By.TAG_NAME, 'table').find_elements(By.TAG_NAME,'tr')]
rows

[[],
 ['1',
  '',
  'Carlsen, Magnus',
  '',
  '2838.4',
  '+3.4',
  '',
  '2825.0',
  '',
  '2887.0',
  '',
  '',
  '32',
  ''],
 ['2',
  '↑1',
  'Caruana, Fabiano',
  '',
  '2782.6',
  '+0.6',
  '',
  '2763.0',
  '',
  '2813.0',
  '',
  '',
  '31',
  ''],
 ['3',
  '↓1',
  'Nakamura, Hikaru',
  '',
  '2782.6',
  '−4.4',
  '',
  '2744.8',
  '',
  '2874.0',
  '',
  '',
  '35',
  ''],
 ['4',
  '',
  'Ding, Liren',
  '',
  '2780.0',
  '0.0',
  '',
  '2830.0',
  '',
  '2787 i',
  '',
  '',
  '30',
  ''],
 ['5',
  '↑1',
  'Firouzja, Alireza',
  '',
  '2777.0',
  '0.0',
  '',
  '2742.0',
  '',
  '2896.0',
  '',
  '',
  '20',
  ''],
 ['6',
  '↓1',
  'Nepomniachtchi, Ian',
  '',
  '2774.8',
  '−4.2',
  '',
  '2762.0',
  '',
  '2785.0',
  '',
  '',
  '33',
  ''],
 ['7',
  '↑1',
  'So, Wesley',
  '',
  '2760.6',
  '−8.4',
  '',
  '2787.8',
  '',
  '2765.0',
  '',
  '',
  '29',
  ''],
 ['8',
  '↓1',
  'Giri, Anish',
  '',
  '2759.6',
  '−9.4',
  '',
  '2705.4',
  '',
  '2762.0',
  '',
  '',
  '29

In [15]:
table = pd.DataFrame(rows)
table.columns = [i.text for i in driver.find_element(By.TAG_NAME, 'table').find_elements(By.TAG_NAME,'tr')[0].find_elements(By.TAG_NAME,'th')]
driver.quit()
table

,#,↑↓,Name,,Classic,+/−,,Rapid,,Blitz,,,Age,
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1,,"Carlsen, Magnus",,2838.4,+3.4,,2825.0,,2887.0,,,32,
2,2,↑1,"Caruana, Fabiano",,2782.6,+0.6,,2763.0,,2813.0,,,31,
3,3,↓1,"Nakamura, Hikaru",,2782.6,−4.4,,2744.8,,2874.0,,,35,
4,4,,"Ding, Liren",,2780.0,0.0,,2830.0,,2787 i,,,30,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,,"Najer, Evgeniy",,2648.0,0.0,,2579.0,,2545.0,,,46,
97,97,↓14,"Saric, Ivan",,2647.5,−9.5,,2631.0,,2586.0,,,32,
98,98,,"Malakhov, Vladimir",,2646.0,0.0,,2666.0,,2652.0,,,42,
99,99,↓4,"Liang, Awonder",,2645.6,−3.4,,2518.4,,2538.0,,,20,


In [16]:
column_indices = [1, 3, 5, 6, 8, 10, 11, 13]
table = table.drop(table.columns[column_indices], axis=1)
table

,#,Name,Classic,Rapid,Blitz,Age
0,None,None,None,None,None,None
1,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32
2,2,"Caruana, Fabiano",2782.6,2763.0,2813.0,31
3,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35
4,4,"Ding, Liren",2780.0,2830.0,2787 i,30
...,...,...,...,...,...,...
96,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46
97,97,"Saric, Ivan",2647.5,2631.0,2586.0,32
98,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42
99,99,"Liang, Awonder",2645.6,2518.4,2538.0,20


In [17]:
table = table.drop(table.index[0])
table

,#,Name,Classic,Rapid,Blitz,Age
1,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32
2,2,"Caruana, Fabiano",2782.6,2763.0,2813.0,31
3,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35
4,4,"Ding, Liren",2780.0,2830.0,2787 i,30
5,5,"Firouzja, Alireza",2777.0,2742.0,2896.0,20
...,...,...,...,...,...,...
96,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46
97,97,"Saric, Ivan",2647.5,2631.0,2586.0,32
98,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42
99,99,"Liang, Awonder",2645.6,2518.4,2538.0,20


### Crear una lista con los jugarores

In [32]:
top_100 = table['Name'].tolist()
print(top_100)


['Carlsen, Magnus', 'Caruana, Fabiano', 'Nakamura, Hikaru', 'Ding, Liren', 'Firouzja, Alireza', 'Nepomniachtchi, Ian', 'So, Wesley', 'Giri, Anish', 'Gukesh D', 'Anand, Viswanathan', 'Rapport, Richard', 'Karjakin, Sergey', 'Dominguez Perez, Leinier', 'Radjabov, Teimour', 'Aronian, Levon', 'Mamedyarov, Shakhriyar', 'Le, Quang Liem', 'Grischuk, Alexander', 'Duda, Jan-Krzysztof', 'Vachier-Lagrave, Maxime', 'Topalov, Veselin', 'Wei, Yi', 'Yu, Yangyi', 'Vitiugov, Nikita', 'Vidit, Santosh Gujrathi', 'Abdusattorov, Nodirbek', 'Praggnanandhaa R', 'Keymer, Vincent', 'Vallejo Pons, Francisco', 'Harikrishna, Pentala', 'Dubov, Daniil', 'Maghsoodloo, Parham', 'Sjugirov, Sanan', 'Erigaisi Arjun', 'Deac, Bogdan-Daniel', 'Robson, Ray', 'Wang, Hao', 'Van Foreest, Jorden', 'Artemiev, Vladislav', 'Sevian, Samuel', 'Shankland, Sam', 'Eljanov, Pavel', 'Tomashevsky, Evgeny', 'Xiong, Jeffery', 'Sargissian, Gabriel', 'Nihal Sarin', 'Svidler, Peter', 'Bu, Xiangzhi', 'Sarana, Alexey', 'Navara, David', 'Anton Gui

# Scrapeo de datos de 'Yottachess.com'

In [18]:
# ocultar ser un robot

from selenium.webdriver.chrome.options import Options

# opciones del driver
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

In [19]:
# entrando en la url

url = 'https://www.yottachess.com/'

driver = webdriver.Chrome(PATH, options=opciones)

driver.get(url)

time.sleep(2)

In [20]:
#aceptar cookies

aceptar = driver.find_element(By.XPATH, '//*[@id="ez-accept-necessary"]')

aceptar.click()

time.sleep(2)

In [21]:
# introducir texto de busqueda, (habrá que hacer un loop para ir metiendo los nombres del top 100)

texto = driver.find_element(By.XPATH, '//*[@id="data"]')

texto.send_keys('Carlsen, Magnus')

time.sleep(2)

In [22]:
texto.send_keys(Keys.ENTER)   # aprieta el boton enter del teclado

### intento coger la tabla entera, (no ha funcionado por lo que cogerésolamete los datos que me interesan)

In [23]:
wins = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[9]/div/h6').text
wins

'2879 (48.53 %)'

In [24]:
draw = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[10]/div/h6').text
draw

'718 (35.27 %)'

In [25]:
loss = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[11]/div/h6').text
loss

'961 (16.2 %)'

In [26]:
driver.quit()

In [27]:
# decorador, solo si haces de forma asíncrona, pero yo voy a paralelizar
def asincrono(funcion):
    
    def eventos(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, funcion, *args, **kwargs)
    
    return eventos

# Lo que hay que hacer (esquema)

In [33]:
# crea un codigo que haga lo que te pido a continuación:

# entrar en la url

url = 'https://www.yottachess.com/'

driver = webdriver.Chrome(PATH, options=opciones)

driver.get(url)

time.sleep(2)


#aceptar cookies

aceptar = driver.find_element(By.XPATH, '//*[@id="ez-accept-necessary"]')

aceptar.click()

# introducir texto de busqueda, (habrá que hacer un loop para ir metiendo los nombres del top 100, los cuales estarán almacenados en una lista llamada 'top_100')

texto = driver.find_element(By.XPATH, '//*[@id="data"]')

texto.send_keys('Carlsen, Magnus')

# aprieta el boton enter del teclado

from selenium.webdriver.common.keys import Keys

texto.send_keys(Keys.ENTER)

#obtiene el porcentaje de victorias

wins = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[9]/div/h6').text
wins

#obtiene el porcentaje de empates

draws = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[10]/div/h6').text
draws

#obtiene el porcentaje de derrotas

losses = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[11]/div/h6').text
losses

driver.quit()

# almacenar en un dataframe los datos obtenidos para cada jugador, es decir, tendrá que tener las siguentes columnas: nombre del jugador, wins, draws, losses (obtenidas en el paso anterior)

# realizar este proceso paralelizando con joblib.

# let's fucking go

In [ ]:
# Función para obtener las estadísticas de un jugador
def obtener_estadisticas(jugador):
    driver = webdriver.Chrome(PATH, options=opciones)
    driver.get(url)
    time.sleep(2)

    # Aceptar cookies
    aceptar = driver.find_element(By.XPATH, '//*[@id="ez-accept-necessary"]')
    aceptar.click()
    
    try:
        # Introducir el nombre del jugador
        texto = driver.find_element(By.XPATH, '//*[@id="data"]')
        texto.send_keys(jugador)
        texto.send_keys(Keys.ENTER)
        time.sleep(2)

        # Obtener las estadísticas
        wins = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[9]/div/h6').text
        draws = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[10]/div/h6').text
        losses = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[11]/div/h6').text

        driver.quit()  # Cerrar el navegador
        
    except:
        time.sleep(1)

    return {'Jugador': jugador, 'Wins': wins, 'Draws': draws, 'Losses': losses}

# URL y opciones del driver
url = 'https://www.yottachess.com/'
PATH = 'ruta_al_chromedriver'
opciones = webdriver.ChromeOptions()

# Lista de jugadores top 100
# top_100 ya creada

# Realizar el procesamiento en paralelo para obtener las estadísticas de los jugadores
paralelo = Parallel(n_jobs=3,  verbose=True)
resultados = paralelo(n_jobs=-1)(delayed(obtener_estadisticas)(jugador) for jugador in top_100)

# Crear el DataFrame con los resultados
df = pd.DataFrame(resultados)

# Mostrar el DataFrame
print(df)